Install Packages

In [9]:
%pip install pandas
%pip install mlxtend

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/selenium-3.141.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/selenium-3.141.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


Importing Libraries

In [14]:
#Importing libraries
import pandas as pd
import itertools
from collections import defaultdict
import time
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from IPython.display import display, HTML

Initializing path for datasets


In [15]:
# Dictionary of available datasets
datasets = {
        1: 'data/Amazon.csv',  # Amazon
        2: 'data/Costco.csv',  # Costco
        3: 'data/Levis.csv',  # Levis
        4: 'data/Nike.csv',  # Nike
        5: 'data/Walmart.csv'   # Walmart
    }

Loading the dataset and performing preprocessing operations

In [16]:
# Load dataset from CSV file
def load_data(file_path):
    df = pd.read_csv(file_path)
    transactions = df.apply(lambda x: set(x.dropna().astype(str)), axis=1).tolist()  # Ensure all items are strings
    return transactions

Displaing data from dataset

In [17]:
# Function to load data using pandas
def load_data_display(file_path):
    try:
        # Load the CSV file using pandas
        data = pd.read_csv(file_path)
        print(f"Data loaded successfully from {file_path}")
        return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return None

# Function to display items and transactions in table format with borders
def display_items_and_transactions(data):
    # Combine all item columns into one list for unique items
    items = pd.concat([data['Item 1'], data['Item 2'], data['Item 3'], data['Item 4']]).dropna().unique()
    
    # Create a DataFrame for items
    items_df = pd.DataFrame(sorted(items), columns=["Items"])
    items_df.index = range(1, len(items_df) + 1)  # Adjust the index to start from 1
    print("\n--- List of Items in All Transactions ---")
    
    # Display items with borders
    display(HTML(items_df.to_html(border=1)))

    # Create a DataFrame for transactions
    transactions_df = data[['Transaction ID', 'Item 1', 'Item 2', 'Item 3', 'Item 4']].copy()
    transactions_df = transactions_df.set_index('Transaction ID')  # Keep the default index for transactions
    print("\n--- List of Transactions ---")
    
    # Display transactions with borders
    display(HTML(transactions_df.to_html(border=1)))

list_data = ['Amazon', 'Costco', 'Levis', 'Nike', 'Walmart']

# Loop until valid input or exit
while True:
    print("Please choose a dataset:")
    for key, name in zip(datasets.keys(), list_data):
        print(f"{key}. {name}")
    print("6. Exit")  # Option to exit

    try:
        choice = int(input("Enter the number corresponding to the dataset: "))
        if choice == 6:
            print("Exiting program.")
            break  # Exit the program
        elif choice in datasets:
            print(f"\nDisplaying Items and Transactions for: {list_data[choice-1]}\n")
            file_path = datasets[choice]
            print(f"Loading data from {file_path}")
            transactions = load_data_display(file_path)
            if transactions is not None:
                display_items_and_transactions(transactions)  # Display items and transactions in table format
                break  # Valid dataset chosen and loaded, exit the loop
        else:
            print("Invalid choice. Please select a valid dataset.")
    except ValueError:
        print("Invalid input. Please enter a number corresponding to the dataset.")


Please choose a dataset:
1. Amazon
2. Costco
3. Levis
4. Nike
5. Walmart
6. Exit

Displaying Items and Transactions for: Nike

Loading data from data/Nike.csv
Data loaded successfully from data/Nike.csv

--- List of Items in All Transactions ---


,Items
1,Backpack
2,Fitness Leggings
3,Headband
4,Jacket
5,Running Shoes
6,Soccer Ball
7,Socks
8,Sports Bra
9,T-Shirt
10,Water Bottle



--- List of Transactions ---


,Item 1,Item 2,Item 3,Item 4
Transaction ID,,,,
1,Running Shoes,Sports Bra,NaN,NaN
2,Jacket,Backpack,Water Bottle,Soccer Ball
3,Headband,Socks,Running Shoes,Fitness Leggings
4,T-Shirt,Sports Bra,Backpack,NaN
5,Fitness Leggings,Jacket,Soccer Ball,Headband
6,Running Shoes,Water Bottle,T-Shirt,NaN
7,Sports Bra,Headband,Backpack,NaN
8,Jacket,T-Shirt,Water Bottle,Soccer Ball
9,Fitness Leggings,Socks,Sports Bra,NaN


Brute Force Approach

In [18]:
# Generate 1-itemsets for Brute Force
def generate_1_itemsets(transactions):
    itemset = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            itemset[item] += 1
    return {frozenset([item]): count for item, count in itemset.items()}

# Generate k-itemsets from (k-1)-itemsets
def generate_k_itemsets(prev_itemsets, k):
    new_itemsets = set()
    prev_itemsets = list(prev_itemsets.keys())
    
    for i in range(len(prev_itemsets)):
        for j in range(i + 1, len(prev_itemsets)):
            candidate = prev_itemsets[i] | prev_itemsets[j]
            if len(candidate) == k:
                new_itemsets.add(candidate)
    
    return new_itemsets

# Count support for itemsets
def count_support(transactions, itemsets):
    itemset_count = defaultdict(int)
    
    for transaction in transactions:
        for itemset in itemsets:
            if itemset.issubset(transaction):
                itemset_count[itemset] += 1
                
    return itemset_count

# Generate frequent itemsets for Brute Force
def generate_frequent_itemsets(transactions, min_support):
    frequent_itemsets = {}
    total_transactions = len(transactions)
    k = 1

    # Generate 1-itemsets
    current_itemsets = generate_1_itemsets(transactions)

    while current_itemsets:
        # Filter itemsets based on min_support
        frequent_itemsets_k = {itemset: count for itemset, count in current_itemsets.items() if (count / total_transactions) * 100 >= min_support}
        if not frequent_itemsets_k:
            break
        
        # # Print the current k-itemsets and their counts
        # print(f"\n{k}-itemsets:")
        # for itemset, count in frequent_itemsets_k.items():
        #     support = (count / total_transactions) * 100
        #     print(f"Itemset: {set(itemset)}, Count: {count}, Support: {support:.2f}%")
        
        frequent_itemsets.update(frequent_itemsets_k)
        
        # Generate next k-itemsets
        k += 1
        current_candidates = generate_k_itemsets(frequent_itemsets_k, k)
        current_itemsets = count_support(transactions, current_candidates)
    
    return frequent_itemsets

# Generate association rules from frequent itemsets
def generate_association_rules(frequent_itemsets, min_confidence, total_transactions):
    rules = []
    
    for itemset, support_count in frequent_itemsets.items():
        for i in range(1, len(itemset)):
            for antecedent in itertools.combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                
                if consequent:
                    confidence = (support_count / frequent_itemsets[antecedent]) * 100
                    support = (support_count / total_transactions) * 100
                    if confidence >= min_confidence:
                        rules.append((antecedent, consequent, support, confidence))
    
    return rules

# Encode transactions to one-hot format
def encode_transactions(transactions):
    items = sorted(set(item for transaction in transactions for item in transaction))
    encoded_df = pd.DataFrame(0, index=range(len(transactions)), columns=items)
    
    for idx, transaction in enumerate(transactions):
        for item in transaction:
            encoded_df.at[idx, item] = 1
    
    return encoded_df

import time

Extracting and Comparing Results

In [19]:
# Function to transform rules into a consistent format for comparison (antecedents, consequents, support, confidence)
def format_rules(rule_list, brute_force=False):
    formatted_rules = set()
    if brute_force:
        # Brute force rules already come as tuples (antecedent, consequent, support, confidence)
        for antecedent, consequent, support, confidence in rule_list:
            formatted_rules.add((frozenset(antecedent), frozenset(consequent), round(support, 2), round(confidence, 2)))
    else:
        # For Apriori and FP-Growth, rules are in DataFrame format with 'antecedents', 'consequents', 'support', 'confidence'
        for _, row in rule_list.iterrows():
            formatted_rules.add((frozenset(row['antecedents']), frozenset(row['consequents']), round(row['support'] * 100, 2), round(row['confidence'] * 100, 2)))
    return formatted_rules

# Function to compare the association rules across algorithms
def compare_algorithm_rules(brute_force_rules, apriori_rules, fp_growth_rules):
    # Format the rules from each approach
    brute_force_set = format_rules(brute_force_rules, brute_force=True)
    apriori_set = format_rules(apriori_rules)
    fp_growth_set = format_rules(fp_growth_rules)

    # Check if all sets of rules are identical
    identical_rules = brute_force_set == apriori_set == fp_growth_set

    if identical_rules:
        print("\nThe algorithms produced exactly the same association rules (matching antecedents, consequents, support, and confidence).")
    else:
        print("\nThe algorithms generated differing association rules.")

        # Display specific differences for further analysis
        print("\nRules unique to Brute Force:")
        print(brute_force_set - apriori_set - fp_growth_set)
        
        print("\nRules unique to Apriori:")
        print(apriori_set - brute_force_set - fp_growth_set)
        
        print("\nRules unique to FP-Growth:")
        print(fp_growth_set - brute_force_set - apriori_set)

Main Function

In [20]:
# Assume load_data, generate_frequent_itemsets, generate_association_rules, encode_transactions, apriori, association_rules, and fpgrowth are already defined functions

def main():
    
    # Loop until valid input or exit
    while True:
        print("Please choose a dataset:")
        for key, name in zip(datasets.keys(), ['Amazon', 'Costco', 'Levis', 'Nike', 'Walmart']):
            print(f"{key}. {name}")
        print("6. Exit")  # Option to exit

        try:
            choice = int(input("Enter the number corresponding to the dataset: "))
            if choice == 6:
                print("Exiting program.")
                return  # Exit the program
            elif choice in datasets:
                file_path = datasets[choice]
                print(f"Loading data")
                transactions = load_data(file_path)
                break  # Valid dataset chosen, exit the loop
            else:
                print("Invalid choice. Please select a valid dataset.")
        except ValueError:
            print("Invalid input. Please enter a number corresponding to the dataset.")

    # Input support and confidence thresholds
    min_support = float(input("Enter minimum support threshold (0-100): "))
    print("Support = ",min_support,"%")
    min_confidence = float(input("Enter minimum confidence threshold (0-100): "))
    print("Confidence = ",min_confidence,"%")

    # Brute Force Approach
    start_time = time.time()
    frequent_itemsets_bf = generate_frequent_itemsets(transactions, min_support)
    total_transactions = len(transactions)
    association_rules_bf = generate_association_rules(frequent_itemsets_bf, min_confidence, total_transactions)
    brute_force_time = time.time() - start_time

    # Apriori Approach
    start_time = time.time()
    encoded_df = encode_transactions(transactions)
    frequent_itemsets_apriori = apriori(encoded_df, min_support=min_support / 100, use_colnames=True)
    rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=min_confidence / 100)
    apriori_time = time.time() - start_time

    # FP-Growth Approach
    start_time = time.time()
    frequent_itemsets_fp = fpgrowth(encoded_df, min_support=min_support / 100, use_colnames=True)
    rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=min_confidence / 100)
    fp_growth_time = time.time() - start_time

    # Display results
    print("\nBrute Force Results:")
    for antecedent, consequent, support, confidence in association_rules_bf:
        print(f"Rule: {set(antecedent)} -> {set(consequent)}, Support: {support:.2f}%, Confidence: {confidence:.2f}%")

    print("\nApriori Results:")
    print(rules_apriori[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

    print("\nFP-Growth Results:")
    print(rules_fp[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

    # Comparison of results (calling the comparison function)
    compare_algorithm_rules(association_rules_bf, rules_apriori, rules_fp)


    # Timing performance
    print(f"\nTiming Performance:")
    print(f"Brute Force Time: {brute_force_time:.4f} seconds")
    print(f"Apriori Time: {apriori_time:.4f} seconds")
    print(f"FP-Growth Time: {fp_growth_time:.4f} seconds")

    # Determine which algorithm is fastest
    fastest = min(("Brute Force", brute_force_time), ("Apriori", apriori_time), ("FP-Growth", fp_growth_time), key=lambda x: x[1])
    print(f"\nFastest Algorithm: {fastest[0]} with a time of {fastest[1]:.4f} seconds")


if __name__ == "__main__":
    main()


Please choose a dataset:
1. Amazon
2. Costco
3. Levis
4. Nike
5. Walmart
6. Exit
Loading data
Support =  20.0 %
Confidence =  30.0 %

Brute Force Results:
Rule: {'Smart Speaker'} -> {'Headphones'}, Support: 25.00%, Confidence: 62.50%
Rule: {'Headphones'} -> {'Smart Speaker'}, Support: 25.00%, Confidence: 71.43%
Rule: {'Yoga Mat'} -> {'Kitchen Mixer'}, Support: 20.00%, Confidence: 50.00%
Rule: {'Kitchen Mixer'} -> {'Yoga Mat'}, Support: 20.00%, Confidence: 80.00%
Rule: {'Coffee Maker'} -> {'Fitness Tracker'}, Support: 20.00%, Confidence: 66.67%
Rule: {'Fitness Tracker'} -> {'Coffee Maker'}, Support: 20.00%, Confidence: 57.14%
Rule: {'Wireless Mouse'} -> {'Smart Speaker'}, Support: 30.00%, Confidence: 60.00%
Rule: {'Smart Speaker'} -> {'Wireless Mouse'}, Support: 30.00%, Confidence: 75.00%
Rule: {'Wireless Mouse'} -> {'Headphones'}, Support: 20.00%, Confidence: 40.00%
Rule: {'Headphones'} -> {'Wireless Mouse'}, Support: 20.00%, Confidence: 57.14%
Rule: {'Wireless Mouse'} -> {'Smart Speak

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
